In [ ]:
# Loading the packages
import geopandas as gpd
import requests
import zipfile
import io
!pip install pyinpe -q -U
import pyinpe
from pyinpe.WFS import *

import warnings
warnings.filterwarnings("ignore",category=UserWarning)

In [ ]:
# Downloading a shapefile from url,
# unzipping and printing the file names
url = "https://www.gov.br/icmbio/pt-br/assuntos/dados_geoespaciais/mapa-tematico-e-dados-geoestatisticos-das-unidades-de-conservacao-federais/limites_ucs_federais_22102024_a.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
print(filenames)

In [ ]:
# Get a geodataframe with alerts using the IBGE geocode for the city of Brasilândia, Mato Grosso do Sul
getAlerts(spatial_filter=5002308) 

In [ ]:
# Reading the shapefile into a geopandas GeoDataFrame
gdf = gpd.read_file('limites_ucs_federais_22102024_a.shp')
gdf

In [ ]:
# Selecting a polygon from the shapefile
# (in this example, it's a conservation unit 
# in Tocantins, Brazil)
gdf_uc = gdf[gdf.UFAbrang == 'TO'].reset_index(drop=True)
polygon = gdf_uc.geometry[1]
polygon

In [ ]:
# Connecting to DETER Cerrado WFS
connectDeter('Cerrado')

# Getting the alerts
df_alerts = getAlerts(spatial_filter = polygon.wkt, temporal_filter=['2018-01-01', '2021-12-31'])
df_alerts

In [ ]:
# Connecting to Queimadas WFS
connectQueimadas()
# Getting the fires
df_fires = getFires(spatial_filter = polygon.wkt, temporal_filter=['2018-01-01', '2021-12-31'])
df_fires

In [ ]:
# Saving as CSV
gdf_uc.to_file('polygon.csv')
df_alerts.to_csv('df_alerts.csv')
df_fires.to_csv('df_fires.csv')

# Saving as SHP
gdf_uc.to_file('polygon.shp')
df_alerts.to_file('df_alerts.shp') 
df_fires.to_file('df_fires.shp')  